In [1]:
import time 
import random
import numpy as np
from futu import *
import pandas as pd 
from queue import Queue
from quant.data.sqlitedatahandler import SQLiteDataHandler
from quant.backtest import Backtest

In [2]:
ktype = SubType.K_DAY
autype = AuType.QFQ
futu_host = '127.0.0.1'
futu_port = 11111
quote_ctx: OpenQuoteContext = OpenQuoteContext(host=futu_host, port=futu_port)

2020-06-27 21:05:08,534 [open_context_base.py] _socket_reconnect_and_wait_ready:257: Start connecting: host=127.0.0.1; port=11111;
2020-06-27 21:05:08,535 [open_context_base.py] on_error:367: Connect error: conn_id=1; err=ECONNREFUSED;
2020-06-27 21:05:08,537 [open_context_base.py] on_error:367: Connect error: conn_id=1; err=Connection closed;
2020-06-27 21:05:08,539 [open_context_base.py] on_error:367: Connect error: conn_id=1; err=Connection closed;
2020-06-27 21:05:08,540 [network_manager.py] work:366: Close: conn_id=1
2020-06-27 21:05:08,547 [open_context_base.py] _wait_reconnect:475: Wait reconnect in 8 seconds: host=127.0.0.1; port=11111;
2020-06-27 21:05:16,548 [open_context_base.py] _socket_reconnect_and_wait_ready:257: Start connecting: host=127.0.0.1; port=11111;
2020-06-27 21:05:16,554 [open_context_base.py] on_error:367: Connect error: conn_id=2; err=ECONNREFUSED;
2020-06-27 21:05:16,556 [open_context_base.py] on_error:367: Connect error: conn_id=2; err=Connection closed;
2

In [3]:
plates = [('HK.BK1107', '蓝筹股'), ('HK.BK1058', '电商及互联网'), ('HK.BK1117', '互联网+'), ('HK.BK1116', '医疗保健'), 
          ('HK.BK1013', '半导体'), ('HK.BK1050', '生物科技'), ('HK.BK1175', '一带一路'), ('HK.BK1176', '5G概念'), 
          ('HK.BK1178', '粤港澳大湾区'), ('HK.BK1188', '抗疫概念'), ('HK.BK1238', '蓝筹地产股'), ('HK.BK1252', '香港银行股'),
          ('HK.BK1004', '信贷'), ('HK.BK1040', '汽车'), ('HK.BK1217', '汽车股'), ('HK.BK1050', '生物科技'), 
          ('HK.BK1052', '电子消费品'), ('HK.BK1068', '证券经纪'), ('HK.BK1223', '奢侈品股'), ('HK.BK1984', '富途港股新经济板块'), 
          ('HK.BK1180', '特斯拉概念股'), ('HK.BK1180', '特斯拉概念股'), ('HK.BK1110', '阿里概念股'), ('HK.BK1985', '华为概念'), 
          ('HK.BK1190', '腾讯概念'), ('HK.BK1984', '做空集合股')]
len(plates)

26

In [7]:
stocks = {}
for code, name in plates:
    ret, plate_stocks = quote_ctx.get_plate_stock(plate_code=code)
    assert ret == RET_OK
    print(code, name)
    for row in plate_stocks.iterrows():
        stocks[row[1].code] = (row[1].lot_size, row[1].stock_name)
    time.sleep(3)

len(stocks)

HK.BK1107 蓝筹股
HK.BK1058 电商及互联网
HK.BK1117 互联网+
HK.BK1116 医疗保健
HK.BK1013 半导体
HK.BK1050 生物科技
HK.BK1175 一带一路
HK.BK1176 5G概念
HK.BK1178 粤港澳大湾区
HK.BK1188 抗疫概念
HK.BK1238 蓝筹地产股
HK.BK1252 香港银行股
HK.BK1004 信贷
HK.BK1040 汽车
HK.BK1217 汽车股
HK.BK1050 生物科技
HK.BK1052 电子消费品
HK.BK1068 证券经纪
HK.BK1223 奢侈品股
HK.BK1984 富途港股新经济板块
HK.BK1180 特斯拉概念股
HK.BK1180 特斯拉概念股
HK.BK1110 阿里概念股
HK.BK1985 华为概念
HK.BK1190 腾讯概念
HK.BK1984 做空集合股


449

In [8]:
codes, snapshots = [], []
for code in stocks:
    if len(codes) < 300:
        codes.append(code)
    else:
        codes.append(code)
        ret, snapshot = quote_ctx.get_market_snapshot(codes)
        assert ret == RET_OK
        snapshots.append(snapshot)
        codes = []

if codes:
    ret, snapshot = quote_ctx.get_market_snapshot(codes)
    assert ret == RET_OK
    snapshots.append(snapshot)

snapshots = pd.concat(snapshots).set_index('code')

In [16]:
data = []
for code in stocks:
    lot_size, stock_name = stocks[code]
    row = {
        'code': code,
        'lot_size': lot_size,
        'stock_name': stock_name,
        'total_market_val': snapshots.loc[code, 'total_market_val'],
        'pe_ratio': snapshots.loc[code, 'pe_ratio'],
        'pb_ratio': snapshots.loc[code, 'pb_ratio'],
        'last_price': snapshots.loc[code, 'last_price'],
        
    }
    data.append(row)

data = pd.DataFrame(data).sort_values('total_market_val', ascending=False).set_index('code')
data = data[(data.pe_ratio>0) & (data.total_market_val > 5e9)]
data

,lot_size,stock_name,total_market_val,pe_ratio,pb_ratio,last_price
code,,,,,,
HK.00700,100,腾讯控股,4.633519e+12,44.982,9.254,485.00
HK.09988,100,阿里巴巴-SW,4.605699e+12,28.225,5.577,214.60
HK.01398,1000,工商银行,1.696494e+12,4.937,0.605,4.76
HK.00939,1000,建设银行,1.542568e+12,5.255,0.640,6.17
HK.02318,500,中国平安,1.432257e+12,8.636,1.897,78.35
...,...,...,...,...,...,...
HK.03836,500,和谐汽车,5.716027e+09,10.111,0.714,3.63
HK.01349,1000,复旦张江,5.715640e+09,22.551,5.496,5.48
HK.01515,500,华润医疗,5.601643e+09,13.130,0.868,4.32


In [17]:
ret, plate_list = quote_ctx.get_plate_list(Market.HK, Plate.INDUSTRY)
selected = set(plate_list.code.to_list()) | set(data.index.to_list())
len(selected)

291

In [18]:
list(selected)

['HK.BK1093',
 'HK.01113',
 'HK.00992',
 'HK.BK1078',
 'HK.03333',
 'HK.BK1090',
 'HK.02208',
 'HK.03360',
 'HK.BK1026',
 'HK.01299',
 'HK.02628',
 'HK.01157',
 'HK.00101',
 'HK.00968',
 'HK.06881',
 'HK.00376',
 'HK.01778',
 'HK.BK1028',
 'HK.00763',
 'HK.01349',
 'HK.BK1033',
 'HK.01398',
 'HK.01958',
 'HK.00116',
 'HK.00003',
 'HK.00874',
 'HK.01257',
 'HK.00772',
 'HK.BK1041',
 'HK.03606',
 'HK.03692',
 'HK.BK1098',
 'HK.01038',
 'HK.03396',
 'HK.00939',
 'HK.00144',
 'HK.03380',
 'HK.BK1040',
 'HK.BK1075',
 'HK.00390',
 'HK.BK1058',
 'HK.00152',
 'HK.BK1027',
 'HK.BK1046',
 'HK.02880',
 'HK.01233',
 'HK.03328',
 'HK.03836',
 'HK.00354',
 'HK.BK1031',
 'HK.BK1068',
 'HK.01810',
 'HK.01585',
 'HK.00590',
 'HK.BK1072',
 'HK.BK1021',
 'HK.02186',
 'HK.02858',
 'HK.00151',
 'HK.00696',
 'HK.BK1054',
 'HK.01521',
 'HK.00941',
 'HK.BK1084',
 'HK.00027',
 'HK.BK1025',
 'HK.02356',
 'HK.01776',
 'HK.01381',
 'HK.00780',
 'HK.BK1097',
 'HK.BK1082',
 'HK.02359',
 'HK.03311',
 'HK.00083',
 'H